#**Deploying a simple ML model on AWS services locally using LocalStack**

### Here’s a step-by-step guide for deploying a dummy machine learning model using LocalStack on Windows. This procedure allows you to simulate an AWS environment locally, without needing an AWS account.

---

# Step 1: Install Prerequisites

1. **Install Docker Desktop**

   LocalStack relies on Docker to run services locally.
   - Download Docker Desktop from the [official website](https://www.docker.com/products/docker-desktop/) and install it.
   - After installation, open Docker Desktop and ensure it’s running.

2. **Install Python (if not already installed)**
   - Download Python from the [official Python website](https://www.python.org/downloads/).
   - Ensure that you check the box to "Add Python to PATH" during installation.
   - Verify installation by running:

In [ ]:
     python --version


3. **Install the AWS CLI**
   - Download the AWS CLI version 2 installer from the [AWS CLI official website](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-windows.html).
   - Run the installer, and ensure that the AWS CLI is added to your PATH.
   - Verify installation by running:

In [ ]:
 aws.exe --version

4. **Install LocalStack CLI**
   - Install LocalStack using `pip`:

In [ ]:
pip install localstack


   - Verify installation by running:
    

In [ ]:
localstack --version

# Step 2: Configure LocalStack

1. **Start LocalStack**
   - Open a Command Prompt or PowerShell and start LocalStack:

In [ ]:
localstack start

   - This will start LocalStack with all default services enabled (including S3, Lambda, etc.).

2. **Configure AWS CLI to Work with LocalStack**
   - Set up the AWS CLI with test credentials (LocalStack doesn't validate them):

In [ ]:
aws.exe configure set aws_access_key_id test
aws.exe configure set aws_secret_access_key test
aws.exe configure set default.region us-east-1

# Step 3: Prepare a Dummy Model

1. **Create a Dummy Model File**
   - For this example, we’ll create a simple Python script(using Iris Dataset) to train a dummy model and save it using `pickle`.
   - Create a new file named `train_model.py` with the following content:

In [ ]:
# train.model.py

import os
import pickle
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

# Get the directory of the current script
script_dir = os.path.dirname(os.path.abspath(__file__))

# File path for the output file in the same directory
output_file_path = os.path.join(script_dir, 'model.pkl')

# Load dataset
iris = load_iris()
X, y = iris["data"], iris["target"]

# Train model
model = DecisionTreeClassifier()
model.fit(X, y)

# Save the model
with open(output_file_path, 'wb') as model_file:
    pickle.dump(model, model_file)

print(f"Model trained and saved as {output_file_path}")


   - Run the script to generate `model.pkl`:

In [ ]:
python train_model.py


# Step 4: Upload the Model to S3 in LocalStack

1. **Create an S3 Bucket**
   - Use the AWS CLI to create a bucket in LocalStack:

In [ ]:
aws.exe --endpoint-url=http://localhost:4566 s3 mb s3://my-test-bucket

2. **Upload the Model File to the S3 Bucket**
   - Upload `model.pkl` to the S3 bucket:

In [ ]:
 aws.exe --endpoint-url=http://localhost:4566 s3 cp model.pkl s3://my-test-bucket/model.pkl

# Step 5: Deploy a Lambda Function

1. **Create a Simple Lambda Function**
   - Create a new file named `lambda_function.py` with the following content:

In [ ]:
import pickle
import json
import boto3


# Configure the S3 client to use the LocalStack endpoint
s3 = boto3.client('s3', endpoint_url='http://host.docker.internal:4566')
# s3 = boto3.client('s3', endpoint_url='http://localhost:4566')

def lambda_handler(event, context):
    try:
        print("Attempting to access S3")
        response = s3.get_object(Bucket='my-test-bucket', Key='model.pkl')
        print("File accessed successfully")
        model_data = response['Body'].read()

        model = pickle.loads(model_data)

        return {
            'statusCode': 200,
            'body': json.dumps('Model loaded successfully')
        }

    except Exception as e:
        print(f"Error: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Error: {str(e)}")
        }

2. **Package the Lambda Function**
   - Zip the `lambda_function.py` file to create a deployment package:

In [ ]:
"C:\Program Files\7-Zip\7z.exe" a lambda_function.zip lambda_function.py

3. **Create the Lambda Function in LocalStack**
   - Use the AWS CLI to create a Lambda function:

In [ ]:
aws.exe --endpoint-url=http://localhost:4566 lambda create-function --function-name my-lambda-function --runtime python3.9 --handler lambda_function.lambda_handler --zip-file fileb://lambda_function.zip --role arn:aws:iam::000000000000:role/execution_role


# Step 6: Invoke the Lambda Function

1. **Invoke the Lambda Function**
   - Use the AWS CLI to test the Lambda function:

In [ ]:
aws.exe --endpoint-url=http://localhost:4566 lambda invoke --function-name lambdafunction output.json

- Check the `output.json` file for the result of the prediction in your current directory.

# Step 7: Clean Up (Optional)

1. **Stop LocalStack**
   - You can stop LocalStack by pressing `Ctrl+C` in the terminal where it is running or using:

In [ ]:
     localstack stop


2. **Delete the Created Resources**
   - You can use the AWS CLI commands to delete the S3 bucket and Lambda function if needed.

### Comments:
if needed to update the Lambda function(after some modifings) you can use:

In [ ]:
 aws.exe --endpoint-url=http://localhost:4566 lambda update-function-code --function-name my-lambda-function --zip-file fileb://lambda_function.zip

## These steps should help you deploy a dummy model using LocalStack, simulating an AWS environment locally.

## More explaination above code:


### Step 1: Install Prerequisites (AWS CLI, Docker, LocalStack CLI)
Before you start, you need to have certain tools installed on your local system. AWS CLI is used to interact with AWS services, Docker provides containerized environments for running applications like LocalStack, and the LocalStack CLI lets you manage LocalStack. These tools enable you to simulate an AWS environment on your local machine without needing an actual AWS account.

### Step 2: Configure LocalStack (Start LocalStack, Configure AWS CLI)
LocalStack is a local AWS cloud service emulator. Once you start LocalStack, you need to configure your AWS CLI to communicate with it. This involves setting the endpoint URLs for various AWS services (e.g., S3, Lambda) so that your AWS CLI commands interact with the LocalStack environment instead of the real AWS cloud.

### Step 3: Prepare a Dummy Model (Create a Dummy Model File)
The dummy model file represents a simple version of a machine learning model or any other data file that your Lambda function will use. Since it's a dummy model, it doesn’t perform any complex computations; it can be just a text or JSON file containing basic data or simple algorithms to mimic a real model’s structure.

### Step 4: Upload the Model to S3 in LocalStack (Create an S3 Bucket, Upload the Model File)
To make the dummy model accessible, you create a storage space (bucket) in the LocalStack-simulated S3 service and upload the model file to it. This step mimics storing a model in an actual S3 bucket in AWS, allowing your Lambda function to access the file during execution.

### Step 5: Deploy a Lambda Function (Create a Simple Lambda Function, Package the Lambda Function, Create the Lambda Function in LocalStack)
You need to write a simple Lambda function that can, for example, read the dummy model file from the S3 bucket. The function code is then packaged for deployment. Finally, you create the Lambda function in the LocalStack environment, which simulates the behavior of AWS Lambda.

AWS Lambda is a serverless compute service provided by AWS that allows you to run code in response to specific events without provisioning or managing servers. It automatically scales your application by running code in parallel in response to triggers such as request to an API endpoint, HTTP requests, file uploads to S3, or database changes. You pay only for the compute time used, making it cost-effective for various applications.

In the context of deploying the dummy model, the Lambda function may be designed to load the model from an S3 bucket and perform basic operations, like returning a simple response or processing input data based on the model.

### Step 6: Invoke the Lambda Function
Once the Lambda function is deployed, you can trigger it using the AWS CLI or LocalStack CLI. This invocation allows you to test whether the Lambda function executes as expected in the LocalStack environment, using the model file stored in the simulated S3 service.

### Step 7: Clean Up (Stop LocalStack, Delete the Created Resources)
After testing, you should clean up by stopping LocalStack and deleting the created resources (e.g., the S3 bucket, the Lambda function). This helps free up system resources and prevents conflicts or issues if you need to run LocalStack again for another project.

# **These steps help replicate the process of deploying a machine learning model and serverless functions in a real AWS environment, all within your local development setup.**